# LSTMx4 st12_9_temp +3 LB = 0.95

### Import libs

In [1]:
# Путь расположения файлов, если меняется, то заменить
# Папка с исходными файлами 'X_test.npy', 'X_train.npy', 'y_train.csv', 'sample_submission.csv'
#PATH = '/content/drive/MyDrive/Motorica/'

In [3]:
# Поключение google drive для для чтения файлов
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)
import os
PATH = ""

#print(os.listdir(PATH))

# Папка для сохранения моделей и файлов
PATH_BEST_MODEL = os.path.join(PATH, 'best_models/')

# Импорты необходимых библиотек
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import tensorflow as tf
from keras import backend as K
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

In [4]:
# Загрузка обучающей выборки и меток классов
X_train = np.load(os.path.join(PATH, 'X_train.npy'))

#загрузка тестовой выборки
X_test = np.load(os.path.join(PATH, 'X_test.npy'))

print(f"Размерность файла 'X_train': {X_train.shape}. Размерность: наблюдения, датчики, время")
print(f"Размерность файла 'X_test': {X_test.shape}. Размерность: наблюдения, датчики, время")

y_train = pd.read_csv(os.path.join(PATH, 'y_train.csv'))
print(f"Размерность файла 'y_train': {y_train.shape}")

Размерность файла 'X_train': (323, 40, 100). Размерность: наблюдения, датчики, время
Размерность файла 'X_test': (54, 40, 100). Размерность: наблюдения, датчики, время
Размерность файла 'y_train': (32300, 2)


In [5]:
y_train[['sample_id', 'timestep']] = y_train['sample-timestep'].str.split('-', 1, expand=True).astype(int)
y_train = y_train.pivot(index='sample_id', columns='timestep', values='class')
y_train_index = y_train.index
y_train = y_train.sort_index() # на всякий случай
y_train = y_train.values

print(y_train.shape)
y_train

(323, 100)


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 6, 6, 6],
       ...,
       [0, 0, 0, ..., 7, 7, 7],
       [7, 7, 7, ..., 0, 0, 0],
       [0, 0, 0, ..., 4, 4, 4]], dtype=int64)

### Преобразования y_train
В исходном y_train отражен только момент подачи сигнала. В задаче необходимо по показаниям датчиков предсказать жест (класс) и момент совершения жеста, который совершается позже момента подачи сигнала. Если подать на обучение X_train и y_train без дополнительного преобразования, то даже, если модель обучится очень хорошо (f1-score ~ 1), результат предсказания на тестовой выборке (X_test) в Public Leaderboard будет очень плохим (было получено значение 0.77481). Таким образом, необходимы дополнительные преобразования, для определения момента выполнения жеста по показаниям датчиков. Далее будет кратко описана идея последующих преобразований.

In [8]:
# Смотрим за датчиками и target для второго наблюдения
k=2
import __init__
X_train = __init__.X_train
df0_T = pd.DataFrame(data = X_train[k], index = range(X_train.shape[1]), columns = range(X_train.shape[2])).T
y_k = y_train[k*100:(k+1)*100].reset_index().T
f, (ax1, ax2) = plt.subplots(2, 1, figsize = (10, 8)) 
ax1.plot(list(df0_T.index), df0_T.values)
ax2.plot(list(y_k.columns), y_k.loc['class'].values)
plt.show()

time_signal = y_k.loc['class'][y_k.loc['class'].values<1].index[-1]+1
print(f'Время подачи сигнала = {time_signal}')

AttributeError: 'numpy.ndarray' object has no attribute 'reset_index'

Из рисунков выше видно, что подача сигнала происходит раньше изменения сигналов от датчиков (примерно на 70 временном интервале происходит рост датчиков).  
Для фиксации момента начала роста сигнала датчиков рассмотрим поведение первой производной (np.diff c n=1).

In [ ]:
# Анализируем датчик 8 для рассматриваемого наблюдения 2
time_stp = 0 # начальный интервал времени можно сдвинуть
sensor = 8
df_T_50 = df0_T.iloc[time_stp:]
y_k_50 = y_k[range(time_stp,100)]
f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize = (10, 9))
ax1.plot(list(df_T_50.index), df_T_50[sensor].values)
ax2.plot(list(y_k_50.columns), y_k_50.loc['class'].values)
ax3.plot(list(range(len(np.diff(df_T_50[sensor], 3)))), np.diff(df_T_50[sensor],3))
plt.show()

На рисунке выше видно, что уровень сигнала датчика на временном интервале от 0 до 50 падает, хотя на данном промежутке времени жест зафиксирован. Такое поведение датчиков встречается во многих наблюдениях, что добавляет шум к временной зависимости первой производной и момент роста сигнала датчиков при выполнении жеста можно пропустить либо зафиксировать позже (при выставлении большого порога фиксации первой производной). Было решено искать другие возможности фиксации роста уровней сигналов датчиков.  

Следующей идеей была попытка зафиксировать рост уровня сигналов датчиков с помощью скользящих средних с разными таймфреймами. Две скользящие средние с разными таймфреймами в момент роста сигнала будут пересекаться, можно зафиксировать и интерпретировать, как начало выполнения жеста (Аналог MACD-индикатор в финансовом техническом анализе).  

In [ ]:
k=2
df0_T = pd.DataFrame(data = X_train[k], index = range(X_train.shape[1]), columns = range(X_train.shape[2])).T
y_k = y_train[k*100:(k+1)*100].reset_index().T

# Скользящие средние с одним пересечением
time_stp = 0
time_end = 75
sensor = 0

df_T_50 = df0_T.iloc[time_stp:time_end]
df_T_50_mean_1 = df_T_50.rolling(window=5).mean().dropna(axis=0)
df_T_50_mean_2 = df_T_50.rolling(window=15).mean().dropna(axis=0)
y_k_50 = y_k[range(time_stp, time_end)]


f, (ax1, ax2) = plt.subplots(2, 1, figsize = (10, 10))
ax1.plot(list(df_T_50.index), df_T_50[sensor].values)
ax1.plot(list(df_T_50_mean_1.index), df_T_50_mean_1[sensor].values, color = 'r')
ax1.plot(list(df_T_50_mean_2.index), df_T_50_mean_2[sensor].values, color = 'g')


ax2.plot(list(y_k_50.columns), y_k_50.loc['class'].values)
plt.show()

На графиках выше видно, что идея в некоторых случаях может работать, однако графики, представленные далее, показывают, что пересечений может быть много, что сильно усложнит интерпретацию.

In [ ]:
# Скользящие средние с несколькими пересечениями
time_stp = 0
time_end = 75
sensor = 5

df_T_50 = df0_T.iloc[time_stp:time_end]
df_T_50_mean_1 = df_T_50.rolling(window=5).mean().dropna(axis=0)
df_T_50_mean_2 = df_T_50.rolling(window=15).mean().dropna(axis=0)
y_k_50 = y_k[range(time_stp, time_end)]


f, (ax1, ax2) = plt.subplots(2, 1, figsize = (10, 10)) # , sharey=True
ax1.plot(list(df_T_50.index), df_T_50[sensor].values)
ax1.plot(list(df_T_50_mean_1.index), df_T_50_mean_1[sensor].values, color = 'r')
ax1.plot(list(df_T_50_mean_2.index), df_T_50_mean_2[sensor].values, color = 'g')

ax2.plot(list(y_k_50.columns), y_k_50.loc['class'].values)
plt.show()

Было решено отказаться от использования скользящих средних в связи со сложностью обработки данных с несколькими пересечениями и попробовать следующую идею: расчет среднеквадратичных отклонений для скользящего окна.  
Исходя из определения среднеквадратичного отклонения в случае, если в рассматриваемом окне произойдет рост сигналов, это скажется и на величине среднеквадратичного отклонения (во время выполнения жеста и образования ступеньки показаний датчиков (вверх или вниз) стандартное отклонение будет также расти, на этом можно ловить перепад.  

На рисунке ниже показан временной ряд сигнала датчика 5 (сгиб) для наблюдения 17 (верхний рисунок), скользящее std (средний рисунок). Для сравнения на нижнем рисунке также представлен сигнал выполнения жеста из y_train (нижний рисунок). 
Видно, что идея может также работать, во время непрерывного наблюдения за уровнем сигнала датчика (при параллельном расчете скользящего std). Как только будет превышен некоторый уровень скользящего std, можно считать, что зафиксировано начало выполнения жеста.

In [ ]:
# Жест сгиб
k=17
df0_T = pd.DataFrame(data = X_train[k], index = range(X_train.shape[1]), columns = range(X_train.shape[2])).T
y_k = y_train[k*100:(k+1)*100].reset_index().T

# смотрим скользящее STD
time_stp = 0
time_end = 100
sensor = 5
df_T_50 = df0_T.iloc[time_stp:time_end]
df_T_50_std_1 = df_T_50.rolling(window=5).std().dropna(axis=0)
y_k_50 = y_k[range(time_stp, time_end)]

f, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize = (10, 9)) 
ax1.plot(list(df_T_50.index), df_T_50[sensor].values)
ax2.plot(list(df_T_50_std_1.index), df_T_50_std_1[sensor].values)
ax3.plot(list(y_k_50.columns), y_k_50.loc['class'].values)
plt.show()

Важно отметить, что датчики со слабым уровнем сигнала в скользящем std могут давать сильно зашумленную картину (см. рисунок ниже), что сложно для интерпретации и из анализа подобные датчики будут исключаться.

In [ ]:
# Жест сгиб
k=17
df0_T = pd.DataFrame(data = X_train[k], index = range(X_train.shape[1]), columns = range(X_train.shape[2])).T
y_k = y_train[k*100:(k+1)*100].reset_index().T

# смотрим скользящее STD
time_stp = 0
time_end = 100
sensor = 7
df_T_50 = df0_T.iloc[time_stp:time_end]
df_T_50_std_1 = df_T_50.rolling(window=5).std().dropna(axis=0)
y_k_50 = y_k[range(time_stp, time_end)]

f, (ax1, ax2) = plt.subplots(2, 1, figsize = (10, 6)) 
ax1.plot(list(df_T_50.index), df_T_50[sensor].values)
ax2.plot(list(df_T_50_std_1.index), df_T_50_std_1[sensor].values)
plt.show()

Дополнительно рассмотрим графики скользящих std для наблюдения 2 и датчиков со средним уровнем сигнала больше 200 во временном интервале от 0 до 50. Примечание: в самом верху рисунка представлен график отражающий сигнал к началу выполнения жеста (значение y_train для наблюдения 2).

In [ ]:
# Жест разгиб просмотр всех жатчиков
k=2

df0_T = pd.DataFrame(data=X_train[k], 
                     index=range(X_train.shape[1]), 
                     columns=range(X_train.shape[2])).T

y_k = y_train[k*100:(k+1)*100].reset_index().T

print(f'Измерение {k}')

# строим скользящее STD
time_stp = 0
time_end = 100
df_T_stp = df0_T.iloc[time_stp:time_end]
df_T_stp_std_1 = df_T_stp.rolling(window=5).std().dropna(axis=0)

# Изображаем y_k
y_k_stp= y_k[range(time_stp, time_end)]
f, ax = plt.subplots(1, 1, figsize = (10, 3)) # , sharey=True
ax.plot(list(y_k_stp.columns), y_k_stp.loc['class'].values)
plt.show()

for i in range(df0_T.shape[1]):
    
    # Сразу выделяем датчики со средним больше 200 в интервале 0:50
    if df0_T.iloc[0:50].mean().values[i] > 200:
        sensor = i
        print(f'sensor {sensor}')

        f, (ax1, ax2) = plt.subplots(2, 1, figsize = (10, 10)) # , sharey=True
        ax1.plot(list(df_T_stp.index), df_T_stp[sensor].values)
        ax2.plot(list(df_T_stp_std_1.index), df_T_stp_std_1[sensor].values)
        plt.show()
        print('--------------------------------------------------------------------------')

На основании представленных графиков можно сделать вывод, что не по всем сигналам скользящего std можно однозначно определить, где начался переход, однако это можно учесть дополнительной проверкой или преобразованием данных для уточнения начала ступени датчиков.

Создадим функцию выбора активных / релевантных датчиков по номеру наблюдения.

In [ ]:
def get_sensor_list(id, print_active=False, print_reliable=False):
    
    """
    Функция возвращает номера активных датчиков (абс. значение >200) 
    и номера релевантных (отклонение между абс. средними датчиками в начале и в конце >100).
    Аргумент функции - номер наблюдения и необходимость вывода на экран указанных номеров. 
    """

    global active_sensors, passive_sensors, reliable_sensors, unreliable_sensors, df_mean
    df = pd.DataFrame(data = X_train[id], index = [s for s in range(X_train.shape[1])], 
                      columns = [s for s in range(X_train.shape[2])]
    )

    df_ = {}
    
    # Создадим список индексов активных и пассивных датчиков. 
    # Среднее значение сигнала не превышает 200 единиц.
    active_sensors, passive_sensors  = list(), list()
    reliable_sensors, unreliable_sensors  = list(), list()
    
    df_mean = list() # cписок средних абсолютных амплитуд датчиков

    for i in range(40):
        if df.iloc[i].mean() > 200:
            active_sensors.append(i)
            
            # разница между абсолютными средними значениями за последние 40 сек и первые 60 сек  
            df_[i] =  abs(df.iloc[i][60:].mean() - df.iloc[i][0:59].mean())
            df_mean.append(df_[i])

          # Создадим список индексов надежных и ненадёжных датчиков по амплитуде в 100 единиц
            if df_[i] > 100:
                reliable_sensors.append(i)
            else:
                unreliable_sensors.append(i)
        else:
            passive_sensors.append(i)
    
    if print_active is True:
      if print_reliable is True:
        print(f"Активные датчики наблюдения " + str(id) +": ", active_sensors)
        print(f"Пассивные датчики наблюдения " + str(id) +":", passive_sensors)
        print(f"Датчики с большой амплитудой, наблюдения " + str(id) +": ", reliable_sensors)
        print(f"Датчики с малой амплитудой, " + str(id) +": ", unreliable_sensors) 

    return active_sensors # passive_sensors, reliable_sensors, unreliable_sensors

Для преобразования y_train на основании сигналов датчиков с

In [ ]:
y_train_ch = []
for k in range(X_train.shape[0]):
    
    df0_T = pd.DataFrame(data=X_train[k], 
                         index=range(X_train.shape[1]), 
                         columns=range(X_train.shape[2])).T
    y_k = y_train[k*100:(k+1)*100].reset_index().T.loc['class'].values
    
    # будем изменять только те наблюдения y_train, в которых происходит изменение жеста
    if np.sum(y_k)!=0: 
    # строим скользящее STD
        time_stp = 0
        time_end = 100
        df_T_stp = df0_T.iloc[time_stp:time_end]
        df_T_stp_std_1 = df_T_stp.rolling(window=5).std().dropna(axis=0)
        
        # выбираем активные датчики
        strong_sens_list = get_sensor_list(k) 

        # рассчитываем начало времени выполнения жеста, исходя из медианного значения скользящего std, 
        # для которого превышен порог 15 по каждому активному датчику, 
        # с последующим выбором медианы из полученного списка 
        grow_ind = int(
            np.median([np.median(np.nonzero((df_T_stp_std_1[x]>15).values)[0]) \
                       for x in strong_sens_list \
                       if np.any(np.nonzero((df_T_stp_std_1[x]>15).values)[0])])) + 3

        # преобразование строки y_train:
        y_k_ch = np.empty(100)
        y_k_ch[:grow_ind].fill(y_k[0]) # до grow_ind - заполняем первым значением k-го наблюдения из y_train
        y_k_ch[grow_ind:].fill(y_k[-1]) # после grow_ind - последним значением k-го наблюдения из y_train
        y_train_ch.append(y_k_ch)
    
    else:
        y_k_ch = np.zeros(100)
        y_train_ch.append(y_k_ch)

y_train_ch = np.array(y_train_ch)
print(y_train_ch.shape)

Посмотрим, что получилось после преобразования:

In [ ]:
# Наблюдение 7
i=7 

df0_T = pd.DataFrame(data=X_train[i], 
                         index=range(X_train.shape[1]), 
                         columns=range(X_train.shape[2])).T
df_T_stp = df0_T
df_T_stp_std_1 = df_T_stp.rolling(window=5).std().dropna(axis=0)


fig, axx = plt.subplots(2, 1, figsize=(10,6), sharex=True)
axx[0].plot(pd.DataFrame(df_T_stp_std_1))
axx[1].plot(y_train_ch[i])
axx[1].set_yticks(
                  np.arange(9),
                  ['Open', 'Pinky', 'Ring', 'Middle', 'Pistol', 'Index', 'Thumb', 'OK', 'Grab']
)
    

plt.suptitle(f'Train {i}')
plt.show()

In [ ]:
# Наблюдение 9
i=9
fig, axx = plt.subplots(2, 1, figsize=(10,6), sharex=True)
axx[0].plot(pd.DataFrame(X_train[i].T))
axx[1].plot(y_train_ch[i])
axx[1].set_yticks(
                  np.arange(9),
                  ['Open', 'Pinky', 'Ring', 'Middle', 'Pistol', 'Index', 'Thumb', 'OK', 'Grab']
)
    
plt.suptitle(f'Train {i}')
plt.show()

In [ ]:
# Можно пробежаться по всем 323 наблюдениям X_train и измененного y_train для визуального анализа преобразований
for i in range(X_train.shape[0]):
    fig, axx = plt.subplots(2, 1, figsize=(10,6), sharex=True)
    axx[0].plot(pd.DataFrame(X_train[i].T))
    axx[0].legend()
    axx[1].plot(y_train_ch[i])
    axx[1].set_yticks(
        np.arange(9),
        ['Open', 'Pinky', 'Ring', 'Middle', 'Pistol', 'Index', 'Thumb', 'OK', 'Grab']
    )
    plt.suptitle(f'Train {i}')
    plt.show()

Данные для подачи в модель подготовлены, перейдем к построению модели.   
Для начала рассмотрим Baseline, предложенный организаторами соревнования.


## NN Baseline

### Model architecture baseline

In [ ]:
X_train_nn = X_train.swapaxes(1, 2)
X_test_nn = X_test.swapaxes(1, 2)
y_train_nn = tf.keras.utils.to_categorical(y_train)

print(X_train_nn.shape, y_train_nn.shape)
print(X_test_nn.shape)

In [ ]:
tf.keras.backend.clear_session()

input_channels = x = tf.keras.layers.Input(
    (X_train.shape[2], X_train.shape[1]),
)
x = tf.keras.layers.SimpleRNN(
    units=X_train.shape[2],
    return_sequences=True,
    recurrent_regularizer=tf.keras.regularizers.L1L2(l1=1e-2, l2=1e-3),
    dropout=0.0,
    recurrent_dropout=0.0,
)(x)
x = tf.keras.layers.SimpleRNN(
    units=X_train.shape[2],
    return_sequences=True,
    recurrent_regularizer=tf.keras.regularizers.L1L2(l1=1e-2, l2=1e-3),
    dropout=0.0,
    recurrent_dropout=0.0,
)(x)
output = tf.keras.layers.Dense(units=9, activation='sigmoid')(x) 

model_baseline = tf.keras.Model(
    inputs=input_channels,
    outputs=output,
    name="Model"
)

model_baseline.compile(
    loss="categorical_crossentropy",
    metrics=[
        "accuracy",
    ],
    optimizer=tf.keras.optimizers.Adam(), # learning_rate=1e-2
)
model_baseline.summary()

In [ ]:
history = model_baseline.fit(
    X_train_nn,
    y_train_nn, 
    epochs=200,
    verbose=1
)

### Visualisation

In [ ]:
y_pred_train_nn = model_baseline.predict(X_train_nn)
y_pred_train_nn.shape

In [ ]:
i = 24

fig, axx = plt.subplots(4, 1, sharex=True, figsize=(10, 8))

axx[0].plot(X_train[i].T)
axx[0].set_title('OMG')

axx[1].imshow(y_train_nn[i].T, origin="lower")
axx[1].set_aspect('auto')
axx[1].set_title('Class/Command')
axx[1].set_yticks(
    np.arange(9),
    ['Open', 'Pinky', 'Ring', 'Middle', 'Pistol', 'Index', 'Thumb', 'OK', 'Grab']
)

axx[2].imshow(y_pred_train_nn[i].T, origin="lower") # , vmin=-0.5, vmax=0.5
axx[2].set_aspect('auto')
axx[2].set_title('Model output')
axx[2].set_yticks(
    np.arange(9),
    ['Open', 'Pinky', 'Ring', 'Middle', 'Pistol', 'Index', 'Thumb', 'OK', 'Grab']
)

axx[3].plot(y_pred_train_nn[i].argmax(axis=-1))
axx[3].set_aspect('auto')
axx[3].set_title('Actual Gesture')
axx[3].set_yticks(
    np.arange(9),
    ['Open', 'Pinky', 'Ring', 'Middle', 'Pistol', 'Index', 'Thumb', 'OK', 'Grab']
)
axx[3].set_xlabel('Timestep')

plt.suptitle(f'Train {i}')

plt.tight_layout()

plt.gcf()

### Интерпретация графиков

График 1 - это данные зафиксированные оптомиографическими датчиками. График 2 подача команды на совершение жеста для пилота. Как можно видеть есть некоторый временной лаг, объясняемый временем реакции.

График 3 - это предсказания модели. График 4 - это фактическое движение, на тестовых данных именно оно будет использоваться для скоринга.

### Model predict

In [ ]:
y_pred_nn = model_baseline.predict(X_test_nn).argmax(axis=-1)

print(y_pred_nn.shape)
y_pred_nn

In [ ]:
y_pred = np.concatenate([arr for arr in y_pred_nn])

print(y_pred.shape)
y_pred

In [ ]:
y_test = pd.read_csv(os.path.join(PATH, './sample_submission.csv'))
y_test['class'] = y_pred
y_test.to_csv('./y_test_2rnn_sigmoid_predicted.csv', index=False)
y_test

In [ ]:
y_test['class'].value_counts()

## Model LSTM

In [ ]:
# Подготовка данных для подачи в модель
X_train_nn = X_train.swapaxes(1, 2)
X_test_nn = X_test.swapaxes(1, 2)
y_train_nn = tf.keras.utils.to_categorical(y_train_ch)

print(X_train_nn.shape, y_train_nn.shape)
print(X_test_nn.shape)

In [ ]:
# Функция для расчета метрики f1_score 
# Примечание: Metrics have been removed from Keras core on 2.0 version
# https://stackoverflow.com/questions/66554207/calculating-micro-f-1-score-in-keras

def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """
        Recall metric.
        Only computes a batch-wise average of recall.
        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """
        Precision metric.
        Only computes a batch-wise average of precision.
        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

В baseline было предложено использовать нейронную сеть с двумя слоями SimpleRNN,  На baseline, как и ожидалось был получен довольно низкий скор по метрике f1.  
Для получения лучшего скора и предсказания модели на тестовых данных была собрана модель из 4-х последовательных слоев LSTM (встроенные в Keras слои рекуррентной нейронной сети, предназначенной для предсказания данных временных рядов), одного полносвязанного слоя с функцией активации 'relu' и выходного слоя с функцией активации 'softmax' (для многоклассовой классификации). В качестве loss-функции выбрана "categorical_crossentropy" и optimizer='Adam'. Между выходным и полносвязным слоями добавлены слои BatchNormalization (для ускорения обучения модели и лучшей стабильности) и Dropout (для исключения переобучения).  
При обучении использовались callbacks: для сохранения лучшего результата, постепенного уменьшения learning rate (если не растет f1-score) и остановки обучения в случае длительного отсутствия улучшения рассматриваемой метрики.

In [ ]:
# удаляем предыдущую модель (если был подбор гиперпараметров) и очищаем память
tf.keras.backend.clear_session()

# входной слой
input_channels = x = tf.keras.layers.Input(
    (X_train.shape[2], X_train.shape[1]),
)


x = tf.keras.layers.LSTM(100, return_sequences=True, return_state=True)(x) 
x = tf.keras.layers.LSTM(100, return_sequences=True, return_state=True)(x)  
x = tf.keras.layers.LSTM(100, return_sequences=True, return_state=True)(x)  
x = tf.keras.layers.LSTM(100, return_sequences=True)(x)

x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)


output = tf.keras.layers.Dense(units=9, activation='softmax')(x) #activation=tf.keras.layers.Softmax(axis=-1)

model = tf.keras.Model(
    inputs=input_channels,
    outputs=output,
    name="Model"
)

model.compile(
    loss="categorical_crossentropy",
    metrics=[f1],
    optimizer=tf.keras.optimizers.Adam(), # learning_rate=1e-2
)
model.summary()

# Callbacks that used for training model
def callbacks(lr):
    checkpoint = ModelCheckpoint(PATH_BEST_MODEL+'best_model.hdf5', monitor='f1', 
                                 verbose=1, mode='max', save_best_only=True)
    earlystop = EarlyStopping(monitor='f1', mode='max', patience=300, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='f1', mode='max', factor=0.9, patience=10, verbose=1, 
                                  min_lr=lr/10000)
    return [checkpoint, earlystop, reduce_lr]

callbacks_list = callbacks(lr=0.001)

In [ ]:
tf.keras.utils.plot_model(model, show_shapes=True)

### Model train

In [ ]:
# Обучение модели
history = model.fit(
    X_train_nn,
    y_train_nn, 
    epochs=1500,
    verbose=1,
    callbacks = callbacks_list
)

### Predict

In [ ]:
# Загружаем веса с лучшим результатом и сохраняем на диске
model.load_weights(PATH_BEST_MODEL+'best_model.hdf5')
model.save_weights(PATH_BEST_MODEL+'best_weights_st12_9_temp.hdf5')
model.save(PATH_BEST_MODEL+'best_model_st12_9_temp.hdf5')

In [ ]:
# model.load_weights(PATH_BEST_MODEL+'best_model.hdf5')
y_pred_nn = model.predict(X_test_nn).argmax(axis=-1)

print(y_pred_nn.shape)
y_pred_nn

In [ ]:
y_pred = np.concatenate([arr for arr in y_pred_nn])

print(y_pred.shape)
y_pred

In [ ]:
# Сохраняем результат предсказания модели в отдельный файл
y_test = pd.read_csv(os.path.join(PATH, './sample_submission.csv'))
y_test['class'] = y_pred
y_test.to_csv(PATH_BEST_MODEL+'y_test_4lstm_st12_9_temp_predicted.csv', index=False)
y_test

In [ ]:
y_test['class'].value_counts()